# Azure Cosmos DB No SQL Vector Store

In this notebook we are going to show a quick demo of how to use AzureCosmosDBNoSqlVectorSearch to perform vector searches in LlamaIndex.

If you're opening this Notebook on colab, you will probably need to install LlamaIndex 🦙.

In [8]:
%pip install llama-index-embeddings-openai
%pip install llama-index-llms-azure-openai

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install llama-index

In [12]:
import os
import json
import openai
from llama_index.llms.azure_openai import AzureOpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.azure_openai import AzureOpenAIEmbedding


# Setup Azure OpenAI

The first step is to configure the llm and the embeddings model. These models will be used to create embeddings for the documents loaded into the database and for llm completions.

In [18]:
llm = AzureOpenAI(    
    model="AZURE_OPENAI_MODEL",
    deployment_name="AZURE_OPENAI_DEPLOYMENT_NAME",
    azure_endpoint="AZURE_OPENAI_BASE",
    api_key="AZURE_OPENAI_KEY",
    api_version="AZURE_OPENAI_VERSION",
)

embed_model = AzureOpenAIEmbedding(
    model="AZURE_OPENAI_EMBEDDING_MODEL",
    deployment_name="AZURE_OPENAI_EMBEDDING_MODEL_DEPLOYMENT_NAME",
    azure_endpoint="AZURE_OPENAI_BASE",
    api_key="AZURE_OPENAI_KEY",
    api_version="AZURE_OPENAI_VERSION",
)

In [19]:
from llama_index.core import Settings

Settings.llm=llm
Settings.embed_model=embed_model

# Loading Documents

In this example we will be using the paul_graham essay which will be processed by the SimpleDirectoryReader.

In [20]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=[r"\examples\data\paul_graham\paul_graham_essay.txt"]).load_data()

print("Document ID:", documents[0].doc_id)

Document ID: 02615dc0-7189-4b4a-ad6b-f6ebc44f1364


# Create the index

Here we establish the connection to cosmos db nosql and create a vector store index.

In [21]:
from azure.cosmos import CosmosClient, PartitionKey
from llama_index.vector_stores.azurecosmosnosql import AzureCosmosDBNoSqlVectorSearch
from llama_index.core import StorageContext

#create cosmos client
URL = 'AZURE_COSMOS_CONNECTION_STRING'
KEY = 'AZURE_COSMOS_KEY'
client = CosmosClient(URL, credential=KEY)

#specify vector store properties
indexing_policy = {
    "indexingMode": "consistent",
    "includedPaths": [{"path": "/*"}],
    "excludedPaths": [{"path": '/"_etag"/?'}],
    "vectorIndexes": [{"path": "/embedding", "type": "quantizedFlat"}],
}

vector_embedding_policy = {
    "vectorEmbeddings": [
        {
            "path": "/embedding",
            "dataType": "float32",
            "distanceFunction": "cosine",
            "dimensions": 3072,
        }
    ]
}

partition_key = PartitionKey(path="/id")
cosmos_container_properties_test = {"partition_key": partition_key}
cosmos_database_properties_test = {}

#create vector store
store = AzureCosmosDBNoSqlVectorSearch(cosmos_client=client,
                                       vector_embedding_policy=vector_embedding_policy,
                                       indexing_policy=indexing_policy,
                                       cosmos_container_properties=cosmos_container_properties_test,
                                       cosmos_database_properties=cosmos_database_properties_test,
                                       create_container=True,)

storage_context = StorageContext.from_defaults(vector_store=store)

index = VectorStoreIndex.from_documents(
        documents, storage_context=storage_context
    )


# Query the index
We can now ask questions using our index.

In [22]:
query_engine = index.as_query_engine()
response = query_engine.query("What did the author love working on?")

In [23]:
import textwrap

print(textwrap.fill(str(response), 100))

Before college, the author loved working on writing and programming. They wrote short stories and
tried writing programs on the IBM 1401 in 9th grade. With microcomputers, the author's interest in
programming grew and they wrote simple games, a program to predict how high their model rockets
would fly, and a word processor that their father used to write at least one book.
